In [ ]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()
using WebIO
using Plots

## Animations: Lorenz Attractor

In [ ]:

# define the Lorenz attractor
Base.@kwdef mutable struct Lorenz
    dt::Float64 = 0.02
    σ::Float64 = 10
    ρ::Float64 = 28
    β::Float64 = 8/3
    x::Float64 = 1
    y::Float64 = 1
    z::Float64 = 1
end

function step!(l::Lorenz)
    dx = l.σ * (l.y - l.x);         l.x += l.dt * dx
    dy = l.x * (l.ρ - l.z) - l.y;   l.y += l.dt * dy
    dz = l.x * l.y - l.β * l.z;     l.z += l.dt * dz
end

attractor = Lorenz()


# initialize a 3D plot with 1 empty series
plt = plot3d(
    1,
    xlim = (-30, 30),
    ylim = (-30, 30),
    zlim = (0, 60),
    title = "Lorenz Attractor",
    marker = 2,
)

# build an animated gif by pushing new points to the plot
anim = @animate for i=1:1500
    step!(attractor)
    push!(plt, attractor.x, attractor.y, attractor.z)
end
gif(anim, "anim_fps15.gif", fps = 15)

## Animation with colour: Quantum particle in a box

In [ ]:
using DifferentialEquations, OrdinaryDiffEq, LinearAlgebra, ForwardDiff, Plots; gr()

mutable struct TDSECpu <: Function
    t::Float64              # the last timestep time to calculate Δt
    diff_const::ComplexF64
    ψ::Array{ComplexF64, 2}    # the wavefunction amplitude
    Δψ::Array{ComplexF64, 2}   # place-holder for the Laplacian

    function TDSECpu(ψ0, p)
        self = new()
        ny, nx = size(ψ0)
        self.t = 0.0
        self.diff_const = p
        self.Δψ = zeros(ComplexF64, ny, nx)

        return self
    end
end

In [ ]:

# 5-point stencil
function laplacian(Δψ::Matrix{ComplexF64}, ψ::Matrix{ComplexF64})
    n1, n2 = size(ψ)
    # internal nodes
    for j in 2:n2-1, i in 2:n1-1
        @inbounds Δψ[i,j] = ψ[i+1,j] + ψ[i-1,j] + ψ[i,j+1] + ψ[i,j-1] - 4*ψ[i,j]
    end

    # left/right edges
    for i = 2:n1-1
        @inbounds Δψ[i,1]  = ψ[i+1,1] + ψ[i-1,1] + 2*ψ[i,2] - 4*ψ[i,1]
        @inbounds Δψ[i,n2] = ψ[i+1,n2] + ψ[i-1,n2] + 2*ψ[i,n2-1] - 4*ψ[i,n2]
    end

    # top/bottom edges
    for j = 2:n2-1
        @inbounds Δψ[1,j]  = ψ[1,j+1] + ψ[1,j-1] + 2*ψ[2,j] - 4*ψ[1,j]
        @inbounds Δψ[n1,j] = ψ[n1,j+1] + ψ[n1,j-1] + 2*ψ[n1-1,j] - 4*ψ[n1,j]
    end

    # corners
    @inbounds Δψ[1,1]   = 2*(ψ[2,1] + ψ[1,2]) - 4*ψ[1,1]
    @inbounds Δψ[n1,1]  = 2*(ψ[n1-1,1] + ψ[n1,2]) - 4*ψ[n1,1]
    @inbounds Δψ[1,n2]  = 2*(ψ[2,n2] + ψ[1,n2-1]) - 4*ψ[1,n2]
    @inbounds Δψ[n1,n2] = 2*(ψ[n1-1,n2] + ψ[n1,n2-1]) - 4*ψ[n1,n2]
end
function (f::TDSECpu)(dψ::Matrix{ComplexF64}, ψ::Matrix{ComplexF64}, p, t::Float64)::Matrix{ComplexF64}
    laplacian(f.Δψ, ψ)
    dψ .+= f.diff_const .* f.Δψ
    return dψ
end

In [ ]:
const Nx = 51
const Ny = 51
ψ0 = zeros(ComplexF64, Ny, Nx)
ψ0[div(Ny, 2), div(Nx, 2)] = complex(one(Float64), zero(Float64)) # particle starts in the middle of the box

deriv_cpu = TDSECpu(ψ0, im)

In [ ]:
dψ = similar(ψ0)
laplacian(deriv_cpu.Δψ, ψ0)
@time laplacian(deriv_cpu.Δψ, ψ0)
deriv_cpu(dψ, ψ0, 1, 0.0)
@time deriv_cpu(dψ, ψ0, 1, 0.0);

In [ ]:
using ProgressLogging
condition = function(t, u, integrator)
    true
end
function affect!(integrator)
    nrm = sum(abs2.(integrator.u))
    integrator.u ./= sqrt(nrm)
end
cb = DiscreteCallback(condition,affect!);

In [ ]:
tmax  = 4.0
tspan = (0., tmax)
ψ0    = zeros(ComplexF64, Ny, Nx)
ts    = 0.0:0.01:tmax
ψ0[div(Ny, 2), div(Nx, 2)] = complex(one(Float64), zero(Float64)) # bottom left corner
prob  = ODEProblem(deriv_cpu, ψ0, tspan)
sol   = solve(prob,Tsit5(),maxiters=1e7, abstol=1e-3, reltol=1e-3, progress=true, callback=cb, tstops=ts, saveat=ts);

In [ ]:
@show length(sol)

In [ ]:
tis = filter(i->sol.t[i] ∈ ts, 1:length(sol))
@show length(tis)
anim = @animate for ti in tis
    p1 = surface(1:Nx, 1:Ny, abs2.(sol[ti]), title="t=$(sol.t[ti])")
    p2 = heatmap(1:Nx, 1:Ny, abs2.(sol[ti]), title="t=$(sol.t[ti])")
    plot(p1, p2, layout=(1, 2))
end
gif(anim, "tdse_norm_fps15.gif", fps = 15)